## Fixing the issue with losses not matching in 2 codes


In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize

import pandas as pd

import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml

from scipy import fftpack

In [2]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import spec_loss as spc
import post_analysis_pandas as post

## Read data

In [3]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
img=np.load(ip_fname)[:1000].transpose(0,1,2,3)
t_img=torch.from_numpy(img)
img.shape,t_img.shape

((1000, 1, 128, 128), torch.Size([1000, 1, 128, 128]))

## Spectrum check

Link to use fftpack for pytorch : https://github.com/locuslab/pytorch_fft/issues/9


In [5]:
mean_spec_data,sdev_spec_data=spc.f_torch_image_spectrum(t_img,1)
dict_sample=post.f_compute_hist_spect(img[:,0,:,:],bins=50)

In [6]:
### Test mean 
ans1=mean_spec_data[0,:]
ans2=dict_sample['spec_val']

for i in range(len(ans2)):
    a,b=ans1[i].item(),ans2[i]
    if ((a-b)>1e-2):
        print(i,a,b)


86 37.52268981933594 37.49547916246857


In [7]:
### Test stderr
ans1=sdev_spec_data[0,:]
ans2=dict_sample['spec_err']

for i in range(len(ans2)):
    a,b=ans1[i].item(),ans2[i]
    if ((a-b)>1e-2):
        print(i,a,b)

0 1463.673095703125 1463.3635259438863
1 729.9336547851562 729.7808725265966
2 492.84881591796875 492.7423083249079
3 341.07000732421875 340.9979875539751
4 251.4705047607422 251.4172896418705
5 190.66429138183594 190.62336548128832
6 146.30580139160156 146.27525772497586
7 123.51824951171875 123.49126561776418
8 97.71236419677734 97.69218995694958
9 81.3443832397461 81.32674792307743
10 65.38652038574219 65.37335760784842
11 53.44884490966797 53.43744381703413


### Inference: 
Oct 22, 2020 (Bday!) \
The spectra match. \
The stderr matches less, but it's still in the ballpark. deviation probably due to rounding off errors.

Oct 29, 2022. \
Modified pytorch radial profile code to remove bincount to help with backprop. code taken from tensorflow. \
Results agree!

## Histogram check

In [ ]:
def f_batch_histogram(img_arr,bins,norm,hist_range):
    ''' Compute histogram statistics for a batch of images'''

    ## Extracting the range. This is important to ensure that the different histograms are compared correctly
    if hist_range==None : ulim,llim=np.max(img_arr),np.min(img_arr)
    else: ulim,llim=hist_range[1],hist_range[0]
#         print(ulim,llim)
    ### array of histogram of each image
    hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important
    hist=np.stack(hist_arr[:,0]) # First element is histogram array
#         print(hist.shape)
    bin_list=np.stack(hist_arr[:,1]) # Second element is bin value 
    ### Compute statistics over histograms of individual images
    mean,err=np.mean(hist,axis=0),np.std(hist,axis=0)/np.sqrt(hist.shape[0])
    bin_edges=bin_list[0]
    centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    return mean,err,centers


In [ ]:
def f_compute_hist(data,bins):
    hist_data=torch.histc(data,bins=bins)
    hist_data=(hist_data*bins)/torch.sum(hist_data)
    
    return hist_data

a=f_compute_hist(torch.from_numpy(img),10)
b=f_batch_histogram(img,10,True,None)[0]

print(a,b)

In [ ]:
plt.figure()
plt.plot(a,linestyle='',marker='*')
plt.plot(b,linestyle='',marker='o')

### Test the loss functions on results and keras results

In [ ]:
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/run5_fixed_cosmology/models/gen_imgs.npy'
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201020_130552_spec_loss_/images/gen_img_epoch-23_step-37450.npy'
f1='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201020_130552_spec_loss_/images/gen_img_epoch-19_step-30450.npy'
img_1=np.expand_dims(np.load(f1)[:500],axis=1)

f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/images/gen_img_epoch-2_step-3150.npy'
f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/images/best_-hist_gen_img_epoch-4_step-7002.npy'
f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201015_072548/images/best_-spec_gen_img_epoch-5_step-8012.npy'
f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201020_130552_spec_loss_/images/best-spec_gen_img_epoch-14_step-23290.npy'
f2='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201020_130552_spec_loss_/images/best-hist_gen_img_epoch-2_step-3473.npy'
img_2=np.expand_dims(np.load(f2)[:1000],axis=1)



In [ ]:
mean_spec_data,sdev_spec_data=spc.f_torch_image_spectrum(t_img[:1000],1)
m1,s1=spc.f_torch_image_spectrum(torch.from_numpy(img_1),1)
m2,s2=spc.f_torch_image_spectrum(torch.from_numpy(img_2),1)

In [ ]:
print(spc.loss_spectrum(mean_spec_data,m1,sdev_spec_data,s1,128))
print(spc.loss_spectrum(mean_spec_data,m2,sdev_spec_data,s2,128))

In [ ]:
hist_data=torch.histc(t_img[:1000],bins=50)
print(spc.loss_hist(t_img[5000:7000],hist_data))
print(spc.loss_hist(torch.from_numpy(img_1),hist_data),spc.loss_hist(torch.from_numpy(img_2),hist_data))

### Inference
Oct 20, 2020
There are variations in the spectrum functions in pytorch and numpy.
1. fftshit
2. abs()**2

However, the loss is computed with the same function for input and generated images, so it should not matter.
The comparison with pytorch and keras data shows that keras is doing better (for some reason).